In [2]:
import pandas as pd  # Librería para manejar y analizar datos en forma de tablas
import numpy as np  # Librería para operaciones matemáticas y manejo de arrays
from scipy.stats import t, f_oneway, ttest_ind  # Funciones estadísticas para pruebas de hipótesis y ANOVA

# Cargar los datos desde un archivo Excel
file_path = "Datos_examen_final_19Co2024_Grupo3.xlsx"  # Ruta del archivo Excel con los datos
sheets = ['Santa Ana', 'La Floresta', 'Los Cedros', 'Palermo', 'Córdoba']  # Nombres de las hojas en el archivo

# Leer todas las hojas en un diccionario, donde cada hoja es un DataFrame
data = {sheet: pd.read_excel(file_path, sheet_name=sheet) for sheet in sheets}

# Convertir la columna 'Fecha' en tipo datetime y agregar columna 'Mes'
for df in data.values():
    df['Fecha'] = pd.to_datetime(df['Fecha'], format='%d/%m/%Y')  # Convierte 'Fecha' a formato datetime
    df['Mes'] = df['Fecha'].dt.month  # Extrae el mes de la fecha y lo guarda en una nueva columna 'Mes'

# 1. Cálculo de intervalos de confianza para 'Santa Ana'
santa_ana = data['Santa Ana']  # Seleccionamos el DataFrame de la tienda 'Santa Ana'

# Diccionario para guardar intervalos de confianza
confidence_intervals = {}
alpha_values = [0.05, 0.01]  # Valores de significancia para 95% y 99%
for alpha in alpha_values:
    z = t.ppf(1 - alpha / 2, df=len(santa_ana) - 1)  # Valor crítico de la distribución t de Student
    intervals = {}  # Diccionario para guardar intervalos de cada mes
    for month in santa_ana['Mes'].unique():  # Itera por cada mes único
        monthly_data = santa_ana[santa_ana['Mes'] == month]['Ventas']  # Filtra los datos del mes actual
        mean = monthly_data.mean()  # Calcula la media de las ventas del mes
        std_err = monthly_data.std() / np.sqrt(len(monthly_data))  # Error estándar de la media
        ci = (mean - z * std_err, mean + z * std_err)  # Intervalo de confianza
        intervals[month] = ci  # Guarda el intervalo en el diccionario
    confidence_intervals[alpha] = intervals  # Guarda los intervalos para este nivel de significancia

# 2. Prueba ANOVA para comparar las medias de ventas de todas las tiendas
# Combina las ventas de todas las tiendas en un solo DataFrame, agregando una columna 'Tienda'
ventas_data = pd.concat([df[['Fecha', 'Ventas']].assign(Tienda=tienda) for tienda, df in data.items()])
# Realiza la prueba ANOVA con las ventas de cada tienda
anova_result = f_oneway(*(df['Ventas'] for df in data.values()))

# 3. Identificar la tienda con mayor y menor promedio de ventas
# Calcula el promedio de ventas por tienda
mean_sales = {tienda: df['Ventas'].mean() for tienda, df in data.items()}
max_store = max(mean_sales, key=mean_sales.get)  # Tienda con mayor promedio de ventas
min_store = min(mean_sales, key=mean_sales.get)  # Tienda con menor promedio de ventas

# Prueba de hipótesis para comparar la tienda con mayor promedio y la de menor promedio
max_data = data[max_store]['Ventas']  # Ventas de la tienda con mayor promedio
min_data = data[min_store]['Ventas']  # Ventas de la tienda con menor promedio
t_stat, p_value = ttest_ind(max_data, min_data)  # Prueba t para comparar las medias

# Generar el reporte final
print("**INFORME DE RESULTADOS**\n")

# 1. Reporte de intervalos de confianza
print("1. Intervalos de confianza para el supermercado 'Santa Ana':")
for alpha, intervals in confidence_intervals.items():
    print(f"\nNivel de significancia: {1-alpha:.0%}")  # Nivel de confianza
    for month, ci in intervals.items():
        print(f"  Mes {month}: Intervalo ({ci[0]:.2f}, {ci[1]:.2f})")  # Imprime el intervalo de confianza

# 2. Reporte de ANOVA
print("\n2. Prueba ANOVA para todas las tiendas:")
print(f"  Estadístico F: {anova_result.statistic:.2f}")  # Estadístico F
print(f"  P-valor: {anova_result.pvalue:.4e}")  # Valor p en notación científica
if anova_result.pvalue < 0.05:
    print("  Las diferencias en las ventas entre las tiendas son significativas.")
else:
    print("  No hay evidencia de diferencias significativas entre las tiendas.")

# 3. Reporte de promedios y prueba de hipótesis
print("\n3. Promedios de ventas por tienda:")
for tienda, promedio in mean_sales.items():
    print(f"  {tienda}: {promedio:.2f}")  # Imprime promedio de ventas por tienda

print("\n4. Prueba de hipótesis entre la tienda con mayor y menor promedio:")
print(f"  Tienda con mayor promedio: {max_store} ({mean_sales[max_store]:.2f})")  # Mayor promedio
print(f"  Tienda con menor promedio: {min_store} ({mean_sales[min_store]:.2f})")  # Menor promedio
print(f"  Estadístico t: {t_stat:.2f}")  # Estadístico t de la prueba
print(f"  P-valor: {p_value:.4e}")  # Valor p en notación científica
if p_value < 0.05:
    print("  Las diferencias entre las tiendas son significativas.")
else:
    print("  No hay evidencia suficiente para afirmar que las diferencias entre las tiendas son significativas.")


**INFORME DE RESULTADOS**

1. Intervalos de confianza para el supermercado 'Santa Ana':

Nivel de significancia: 95%
  Mes 1: Intervalo (17000.12, 18765.13)
  Mes 2: Intervalo (19371.88, 21607.56)
  Mes 3: Intervalo (21438.67, 23361.47)
  Mes 4: Intervalo (19754.95, 21851.78)
  Mes 5: Intervalo (21332.79, 22833.65)
  Mes 6: Intervalo (23002.67, 24644.79)
  Mes 7: Intervalo (20766.63, 22735.81)
  Mes 8: Intervalo (22453.12, 24501.65)
  Mes 9: Intervalo (22400.23, 23827.58)
  Mes 10: Intervalo (22234.08, 23930.68)
  Mes 11: Intervalo (21658.40, 23689.87)
  Mes 12: Intervalo (19658.61, 21330.20)

Nivel de significancia: 99%
  Mes 1: Intervalo (16720.58, 19044.67)
  Mes 2: Intervalo (19017.80, 21961.64)
  Mes 3: Intervalo (21134.14, 23666.00)
  Mes 4: Intervalo (19422.86, 22183.87)
  Mes 5: Intervalo (21095.09, 23071.35)
  Mes 6: Intervalo (22742.59, 24904.87)
  Mes 7: Intervalo (20454.76, 23047.69)
  Mes 8: Intervalo (22128.68, 24826.09)
  Mes 9: Intervalo (22174.17, 24053.65)
  Mes 10: I